In [1]:
import pandas as pd
import numpy as np
userdata=pd.read_csv('cleaneddd_user_mapping_following.csv')
userdata.head()

,Unnamed: 0,Unnamed: 0.1,ResearchName,topics,Skills,InterestValue,CiteCount,RecomCount,ItemCount,FollowingCount,FollowerCount,ResearcherURL,Activity,mapping,label_text
0,0,0,Jianbo Bai,"Mechanical Engineering,Heat Transfer,Engineeri...","Modeling and Simulation,Thermal Engineering,He...",152.6,269,0,23,35,30,https://www.researchgate.net/profile/Jianbo_Bai,111.150,1.0,mechanical engineeringheat transferengineering...
1,1,5,Cuicui Ji,"Mechanical Engineering,Materials Engineering,S...","Nonlinear Analysis,Time Series,Fractals,Fracta...",41.6,66,2,17,40,77,https://www.researchgate.net/profile/Cuicui_Ji,31.650,1.0,mechanical engineeringmaterials engineeringstr...
2,2,6,Song Qining,Materials Engineering,"Material Characterization,Materials,Corrosion,...",121.5,106,9,27,39,50,https://www.researchgate.net/profile/Song_Qining,65.875,1.0,materials engineeringmaterial characterization...
3,3,8,Yanqin Shangguan,"Mechanical Engineering,Aerospace Engineering,A...","Fluid Mechanics,Computational Fluid Dynamics,N...",40.1,28,4,10,73,32,https://www.researchgate.net/profile/Yanqin_Sh...,20.525,1.0,mechanical engineeringaerospace engineeringaer...
4,4,13,Imran Dawy,"Electrical Engineering,Artificial Intelligence...","MATLAB Simulation,Control Theory,Robotics,Auto...",1.7,0,0,2,25,8,https://www.researchgate.net/profile/Imran_Dawy,0.925,1.0,electrical engineeringartificial intelligencea...


In [3]:
userdata.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'ResearchName', 'topics', 'Skills',
       'InterestValue', 'CiteCount', 'RecomCount', 'ItemCount',
       'FollowingCount', 'FollowerCount', 'ResearcherURL', 'Activity',
       'mapping', 'label_text'],
      dtype='object')

In [4]:
userdata=userdata[[ 'ResearchName', 'Activity','label_text']]
userdata.head()

,ResearchName,Activity,label_text
0,Jianbo Bai,111.150,mechanical engineeringheat transferengineering...
1,Cuicui Ji,31.650,mechanical engineeringmaterials engineeringstr...
2,Song Qining,65.875,materials engineeringmaterial characterization...
3,Yanqin Shangguan,20.525,mechanical engineeringaerospace engineeringaer...
4,Imran Dawy,0.925,electrical engineeringartificial intelligencea...


In [6]:
from gensim.models import Doc2Vec
textsimdata=pd.read_excel('../program1/user_paper.xlsx')
model = Doc2Vec.load("../program1/Doc2Vec.model")
def to_review_vector(text):
    try:
        words = text.split(" ")
    except:
        words=[text]
    newword=[]
    for i in words:
        try:
            model[i]
            newword.append(i)
        except:
            pass
           
    atemp=np.asarray([model[w] for w in newword ],dtype='float32').mean(axis=0)
    if atemp.shape==():
        print(text)
    else:
         return atemp
train_data_Doc2Vec = [to_review_vector(text) for text in textsimdata['title_abs']]
dic_paper={}
for i in range(textsimdata.shape[0]):
    dic_paper[textsimdata.iloc[i,0].replace('_',' ')]=train_data_Doc2Vec[i]
dic_label={}
train_data_Doc2Vec_label_text = [to_review_vector(text) for text in userdata['label_text']]    
for i in range(userdata.shape[0]):
    dic_label[userdata.iloc[i,0]]=train_data_Doc2Vec_label_text[i]

def paper(_u1, _u2):
    return float(np.sum(_u1*_u2))/(np.linalg.norm(_u1)*np.linalg.norm(_u2))

def label(_u1, _u2):
    return float(np.sum(_u1*_u2))/(np.linalg.norm(_u1)*np.linalg.norm(_u2))

In [7]:
label(dic_label['Jianbo Bai'], dic_label['Cuicui Ji'])

0.5869747116197471

In [8]:
paper(dic_paper['Jianbo Bai'], dic_paper['Cuicui Ji'])

0.9691875605966069

In [11]:
data_following=pd.read_excel('../program2/following_clean.xlsx')
data_following.head()

,ResearcherURL,FollowingURL,Researcher,Following
0,https://www.researchgate.net/profile/Barakat_A...,https://www.researchgate.net/profile/Xinkun_Xu,Barakat Ameen,Xinkun Xu
1,https://www.researchgate.net/profile/Barakat_A...,https://www.researchgate.net/profile/Guoping_Tan,Barakat Ameen,Guoping Tan
2,https://www.researchgate.net/profile/Barakat_A...,https://www.researchgate.net/profile/Xiaoqin_Zeng,Barakat Ameen,Xiaoqin Zeng
3,https://www.researchgate.net/profile/Barakat_A...,https://www.researchgate.net/profile/Yu_Yf,Barakat Ameen,Yu Yf
4,https://www.researchgate.net/profile/Barakat_A...,https://www.researchgate.net/profile/Bing_Shi7,Barakat Ameen,Bing Shi7


In [12]:
all_node=list(set(data_following['Researcher'].values.tolist()  +data_following['Following'].values.tolist()))
all_node_dict={}
for i in all_node:
    temp={}
    
    data_A=data_following[data_following.Researcher==i]
    
    data_B=data_following[data_following.Following==i]

    if data_A.shape[0]!=0:
        for j in list(set(data_A['Following'].unique().tolist())):
            temp[j]=1
    if data_B.shape[0]!=0:
        for k in list(set(data_B['Researcher'].unique().tolist())):
            temp[k]=1
    all_node_dict[i]=temp
def PersonalRank(G, max_sim, root, max_step,u2='feature'):
    alpha=0.85
    rank = dict()
    rank = {x:0 for x in G.keys()}
    rank[root] = 1
    for k in range(max_step):
        tmp = {x:0 for x in G.keys()}
       
        for i, ri in G.items():
            
            for j, wij in ri.items():
               
                tmp[j] += alpha * rank[i] / (1.0 * len(ri))

        tmp[root] += (1 - alpha)
        rank = tmp
    rankvalue=rank.values()
    rankvalue_max=max(rankvalue)
    rankvalue_min=min(rankvalue)
    for key, value in rank.items():
        rank[key]=  (value - rankvalue_min) / (rankvalue_max - rankvalue_min)
    if u2=='feature':
        rank=sorted(rank.items(), key=lambda d:d[1], reverse=True)[1:max_sim+1]
        return rank
    else:
        return rank[u2]
       
        


In [16]:
userdata.head()

,ResearchName,Activity,label_text
0,Jianbo Bai,111.150,mechanical engineeringheat transferengineering...
1,Cuicui Ji,31.650,mechanical engineeringmaterials engineeringstr...
2,Song Qining,65.875,materials engineeringmaterial characterization...
3,Yanqin Shangguan,20.525,mechanical engineeringaerospace engineeringaer...
4,Imran Dawy,0.925,electrical engineeringartificial intelligencea...


In [17]:
all_user=[]
for i in range(userdata.shape[0]):
    all_user.append(userdata.iloc[i,0])


In [18]:
def getu_2_active(username):
    u_2_active=userdata[userdata.ResearchName==username].iloc[0,1]
    return u_2_active


In [19]:
getu_2_active('Jianbo Bai')

111.15

In [20]:
len(all_user)

238

In [21]:
%%time
with open('all_traindata111.csv','a') as f: 
    for i in range(len(all_user)):
        if i%10==0:
            print('进度：{}/238'.format(i))     
        try:
            for k in range(i+1,len(all_user)):
                u1=all_user[i]
                u2=all_user[k]
                Paper_similarity=paper(dic_paper[u1],dic_paper[u2])
                Social_similarity=PersonalRank(G, 10, u1, 5,u2)

                active=getu_2_active(u2)

                ulabel=label(dic_label[u1],dic_label[u2])        
                f.write(str(u1)+','+str(u2)+','+str(Paper_similarity)+','+str(Social_similarity)+','+str(active)+','+str(ulabel)+'\n')
        except:
            pass


进度：0/238
进度：10/238
进度：20/238
进度：30/238
进度：40/238
进度：50/238
进度：60/238
进度：70/238
进度：80/238
进度：90/238
进度：100/238
进度：110/238
进度：120/238
进度：130/238
进度：140/238
进度：150/238
进度：160/238
进度：170/238
进度：180/238
进度：190/238
进度：200/238
进度：210/238
进度：220/238
进度：230/238
Wall time: 8min 39s
